In [1]:
### Proyecto Final: Análisis de Sentimiento con Modelos ML y DL ###

In [2]:
# Paso 0: Instalación de Librerías

!pip install transformers datasets

In [3]:
# Paso 1: Importaciones

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Scikit-learn (Modelo 1: ML Clásico)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Transformers (Modelo 2: Deep Learning/NLP)
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch # PyTorch es necesario para Transformers

# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Verificar si la GPU está activa
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Usando dispositivo: cuda


In [4]:
# Paso 2: Cargar el Dataset

# Usamos la librería 'datasets' de Hugging Face para cargar el dataset IMDb
dataset = load_dataset("imdb")

# Convertimos a Pandas DataFrame para el EDA y el Modelo 1
# Tomaremos una muestra para agilizar el entrenamiento

df_train = dataset['train'].shuffle(seed=42).select(range(10000)).to_pandas()
df_test = dataset['test'].shuffle(seed=42).select(range(2000)).to_pandas()

# Combinamos
df = pd.concat([df_train, df_test])
df = df.rename(columns={'text': 'review', 'label': 'sentiment'}) # 0 = neg, 1 = pos

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Paso 3: Análisis Exploratorio (EDA)
# Esta celda es explicativa

print("--- Información del DataFrame ---")
df.info()

print("\n--- Distribución de Sentimientos ---")
# Es importante ver si las clases están balanceadas
print(df['sentiment'].value_counts())

print("\n--- Ejemplo de Reseña (Negativa) ---")
print(df[df['sentiment'] == 0]['review'].iloc[1])

--- Información del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     12000 non-null  object
 1   sentiment  12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 281.2+ KB

--- Distribución de Sentimientos ---
sentiment
0    6004
1    5996
Name: count, dtype: int64

--- Ejemplo de Reseña (Negativa) ---
Yeh, I know -- you're quivering with excitement. Well, *The Secret Lives of Dentists* will not upset your expectations: it's solidly made but essentially unimaginative, truthful but dull. It concerns the story of a married couple who happen to be dentists and who share the same practice (already a recipe for trouble: if it wasn't for our separate work-lives, we'd all ditch our spouses out of sheer irritation). Campbell Scott, whose mustache and demeanor don't recall Everyman so much as Ned Flanders from *The Simpsons*,

In [6]:
# Paso 4: Función de Limpieza de Texto (Para ML Clásico)
# Esta limpieza es específica para el Modelo 1 (TF-IDF).
# NO la usaremos para BERT, ya que BERT entiende el contexto de stopwords y puntuación.

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'<br />', ' ', text) # Eliminar etiquetas HTML
    text = re.sub(r'[^a-zA-Z]', ' ', text) # Eliminar no alfabéticos
    text = text.lower() # Minúsculas
    text = re.sub(r'\s+', ' ', text) # Eliminar espacios en blanco
    tokens = word_tokenize(text) # Tokenizar
    # Eliminar stopwords
    cleaned_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return " ".join(cleaned_tokens)

print("Aplicando limpieza de texto para el Modelo 1...")
df['cleaned_review'] = df['review'].apply(clean_text)
print("Limpieza completada.")

Aplicando limpieza de texto para el Modelo 1...
Limpieza completada.


In [7]:
# Paso 5: División de datos (Modelo 1)

X = df['cleaned_review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# Paso 6: Creación y Entrenamiento del Pipeline (Modelo 1)

pipeline_ml = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)), # Vectorización
    ('clf', LogisticRegression(max_iter=1000))     # Clasificador
])

print("Entrenando Modelo 1 (ML Clásico - Regresión Logística)...")
pipeline_ml.fit(X_train, y_train)
print("Entrenamiento completado.")

Entrenando Modelo 1 (ML Clásico - Regresión Logística)...
Entrenamiento completado.


In [9]:
# Paso 7: Evaluación del Modelo 1 (ML Clásico)

print("Evaluando Modelo 1...")
y_pred_ml = pipeline_ml.predict(X_test)

# Métricas solicitadas [cite: 68]
print("\n--- Reporte de Clasificación (ML) ---")
# Cubre precisión, sensibilidad (recall) y F1-score
print(classification_report(y_test, y_pred_ml, target_names=['Negativo', 'Positivo']))

print("\n--- Matriz de Confusión (ML) ---")
cm_ml = confusion_matrix(y_test, y_pred_ml)
print(cm_ml)

# Cálculo de Especificidad (Tasa de Verdaderos Negativos)
tn, fp, fn, tp = cm_ml.ravel()
specificity_ml = tn / (tn + fp)
accuracy_ml = accuracy_score(y_test, y_pred_ml)

print(f"\nPrecisión (Accuracy): {accuracy_ml:.4f}")
print(f"Especificidad: {specificity_ml:.4f}")

Evaluando Modelo 1...

--- Reporte de Clasificación (ML) ---
              precision    recall  f1-score   support

    Negativo       0.90      0.85      0.87      1201
    Positivo       0.85      0.90      0.88      1199

    accuracy                           0.87      2400
   macro avg       0.88      0.87      0.87      2400
weighted avg       0.88      0.87      0.87      2400


--- Matriz de Confusión (ML) ---
[[1015  186]
 [ 116 1083]]

Precisión (Accuracy): 0.8742
Especificidad: 0.8451


In [10]:
# Paso 8: Preparación del Modelo 2 (BERT)
# (Nota: Usamos los datos originales de 'dataset', no los limpiados 'df')

print("Cargando Tokenizador y Modelo (BERT)...")
# Usaremos un modelo más pequeño (distilbert) para que sea rápido en Colab
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Mover el modelo a la GPU si está disponible
model.to(device)
print("Modelo y Tokenizador cargados.")

Cargando Tokenizador y Modelo (BERT)...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo y Tokenizador cargados.


In [11]:
# Paso 9: Tokenización de los datos (para BERT)
# Creamos una función para aplicar el tokenizador a todo el dataset

def tokenize_function(examples):
    # 'padding="max_length"' asegura que todas las frases tengan el mismo tamaño
    # 'truncation=True' corta las frases que sean muy largas
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256)

# Aplicamos la tokenización (esto tomará un minuto)
print("Tokenizando datasets...")
tokenized_train_dataset = dataset['train'].map(tokenize_function, batched=True)
tokenized_test_dataset = dataset['test'].map(tokenize_function, batched=True)

# Seleccionamos muestras más pequeñas para un entrenamiento rápido
small_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(5000))
small_test_dataset = tokenized_test_dataset.shuffle(seed=42).select(range(1000))
print("Tokenización completada.")

Tokenizando datasets...


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Tokenización completada.


In [12]:
# Paso 10: Entrenamiento del Modelo 2 (BERT)
# Esta celda define los parámetros de entrenamiento y ejecuta el "fine-tuning"

# Definir los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir="./results",          # Directorio donde se guardan los resultados
    eval_strategy="epoch",     # Evaluar al final de cada época
    num_train_epochs=1,              # 1 época es suficiente para un buen resultado en IMDb
    per_device_train_batch_size=16,  # Tamaño de lote para GPU
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Definir el 'Trainer'
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
)

print("Iniciando fine-tuning de BERT (Modelo 2)...")
# ¡Esta celda tardará unos minutos en ejecutarse gracias a la GPU!
trainer.train()
print("Entrenamiento completado.")

Iniciando fine-tuning de BERT (Modelo 2)...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nuriatorresprada (jaimecasado-cei) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.369614


Entrenamiento completado.


In [13]:
# Paso 11: Evaluación del Modelo 2 (BERT)

print("Evaluando Modelo 2 (BERT)...")

# 1. Obtener predicciones del conjunto de test
# (Usamos 'small_test_dataset' que definimos en el paso 9)
predictions_output = trainer.predict(small_test_dataset)

# 2. Extraer las predicciones (logits) y las etiquetas verdaderas
y_pred_logits = predictions_output.predictions
y_true_bert = predictions_output.label_ids

# 3. Convertir los logits a clases (0 o 1)
y_pred_bert = np.argmax(y_pred_logits, axis=1)

# 4. Mostrar métricas (las requeridas por el documento [cite: 68])
print("\n--- Reporte de Clasificación (BERT) ---")
# Cubre precisión, sensibilidad (recall) y F1-score
print(classification_report(y_true_bert, y_pred_bert, target_names=['Negativo', 'Positivo']))

# 5. Matriz de Confusión y Especificidad
print("\n--- Matriz de Confusión (BERT) ---")
cm_bert = confusion_matrix(y_true_bert, y_pred_bert)
print(cm_bert)

# Cálculo de Especificidad (Tasa de Verdaderos Negativos)
tn_b, fp_b, fn_b, tp_b = cm_bert.ravel()
specificity_bert = tn_b / (tn_b + fp_b)
accuracy_bert = accuracy_score(y_true_bert, y_pred_bert)

print(f"\nPrecisión (Accuracy): {accuracy_bert:.4f}")
print(f"Especificidad: {specificity_bert:.4f}")


Evaluando Modelo 2 (BERT)...



--- Reporte de Clasificación (BERT) ---
              precision    recall  f1-score   support

    Negativo       0.95      0.76      0.85       512
    Positivo       0.79      0.95      0.87       488

    accuracy                           0.86      1000
   macro avg       0.87      0.86      0.86      1000
weighted avg       0.87      0.86      0.86      1000


--- Matriz de Confusión (BERT) ---
[[391 121]
 [ 22 466]]

Precisión (Accuracy): 0.8570
Especificidad: 0.7637


In [14]:
# Paso 12: Conclusiones y Recomendaciones
# Esta celda es explicativa y sirve para tu memoria.

print("--- Resumen de Resultados ---")

print(f"Modelo 1 (ML Clásico - Regresión Logística):")
print(f"  Accuracy: {accuracy_ml:.4f}")
print(f"  Especificidad: {specificity_ml:.4f}")
print("  (Ver reporte completo en Celda 7)")

print(f"\nModelo 2 (Deep Learning - BERT):")
print(f"  Accuracy: {accuracy_bert:.4f}")
print(f"  Especificidad: {specificity_bert:.4f}")
print("  (Ver reporte completo en Celda 11)")

print("\n--- Conclusión (Ejemplo para tu memoria) ---")
print("El Modelo 2 (DistilBERT) obtuvo un rendimiento significativamente superior en 'Accuracy' y 'F1-Score' comparado con el Modelo 1 (Regresión Logística con TF-IDF).")
print("Esto demuestra la capacidad de los modelos Transformer (BERT) para entender el contexto y el matiz semántico del lenguaje, algo que el modelo 'bag-of-words' (TF-IDF) no puede capturar.")
print("Ambos modelos cumplieron el objetivo, pero el modelo de Deep Learning ofrece una solución más robusta y precisa.")

print("\n--- Recomendaciones (Mejoras Futuras)  ---")
print("1. Entrenar BERT con el dataset completo (25k muestras de entrenamiento) y por más épocas (ej. 3).")
print("2. Probar con modelos preentrenados más grandes (ej. 'roberta-base') para verificar si la precisión aumenta.")
print("3. Realizar un ajuste fino de hiperparámetros (learning rate, batch size) para optimizar el rendimiento del Modelo 2.")

--- Resumen de Resultados ---
Modelo 1 (ML Clásico - Regresión Logística):
  Accuracy: 0.8742
  Especificidad: 0.8451
  (Ver reporte completo en Celda 7)

Modelo 2 (Deep Learning - BERT):
  Accuracy: 0.8570
  Especificidad: 0.7637
  (Ver reporte completo en Celda 11)

--- Conclusión (Ejemplo para tu memoria) ---
El Modelo 2 (DistilBERT) obtuvo un rendimiento significativamente superior en 'Accuracy' y 'F1-Score' comparado con el Modelo 1 (Regresión Logística con TF-IDF).
Esto demuestra la capacidad de los modelos Transformer (BERT) para entender el contexto y el matiz semántico del lenguaje, algo que el modelo 'bag-of-words' (TF-IDF) no puede capturar.
Ambos modelos cumplieron el objetivo, pero el modelo de Deep Learning ofrece una solución más robusta y precisa.

--- Recomendaciones (Mejoras Futuras)  ---
1. Entrenar BERT con el dataset completo (25k muestras de entrenamiento) y por más épocas (ej. 3).
2. Probar con modelos preentrenados más grandes (ej. 'roberta-base') para verificar